In [ ]:
import matplotlib.pyplot as plt

#Just for metrics

def pnl(df):
    pred = []
    pos = 0
    profit = 0
    bt = 1000
    actual_loss = 0
    actual_profit = 0
    win = 0
    lose = 0
    mdd = 0
    for index, row in df.iterrows():
        if pos ==1:
          mdd = max(mdd, 1 - df.close[index]*bt/1000)
        elif pos==-1:
          mdd = max(mdd, df.close[index]*1000/bt - 1)
          pass
        if pos < 1 and df.signals[index] == 1:
            pos+=1
            bt /= df.close[index]
        elif pos > -1 and df.signals[index] == -1:
            pos -=1
            bt *= df.close[index]
        else:
            continue
        if pos == 0:
            comm = 0.001*(bt+1000)
            profit += bt - 1000 - comm
            if (bt - 1000 - comm) < 0:
                lose += 1
                actual_loss += abs(bt - 1000 - comm)
            else:
                win += 1
                actual_profit += bt - 1000 - comm
            bt = 1000

            pred.append(profit)
    print("PF: ",actual_profit/actual_loss)
    print("Number of winning trades: ", win)
    print("Number of losing trades: ", lose)
    print("Maximum Drawdown: ", mdd*100)
    return pred


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np


df_hour = pd.read_csv("/content/drive/MyDrive/btcusdt_1h.csv")
#df_hour = pd.read_csv("btcusdt_1h.csv")
df_hour["datetime"] = pd.to_datetime(df_hour["datetime"])
df_hour.set_index("datetime", inplace = True)
df_hour.shape

# Resampling to daily  frequency
df_day = pd.DataFrame()
df_day['open'] = df_hour.resample("D").open.first()
df_day['close'] = df_hour.resample("D").close.last()
df_day['high']=df_hour.resample("D").high.max()
df_day['low'] = df_hour.resample("D").low.min()
df_day['volume'] = df_hour.resample("D").volume.sum()
df_day.reset_index(inplace=True)
df = pd.DataFrame(df_hour)


In [ ]:
def darvas(df, win):
    signals = np.zeros(df.shape[0])
    darvas_up = df.high.rolling(win).max()
    darvas_down = df.low.rolling(win).min()
    pos = 0
    for i in range(win, df.shape[0]):
        if df.close.iloc[i] > 0.95*darvas_up.iloc[i-1]:
            if pos==0:
                signals[i] = 1
                pos = 1
            elif pos==-1:
                signals[i] = 1
                pos = 0
        elif df.close.iloc[i] < 1.05*darvas_down.iloc[i-1]:
            if pos==0:
                signals[i] = -1
                pos = -1
            elif pos==1:
                signals[i] = -1
                pos = 0
    return signals


In [ ]:
df['signals'] = darvas(df, 180)
pnl(df)[-1]

PF:  2.0214781468241765
Number of winning trades:  85
Number of losing trades:  278
Maximum Drawdown:  16.2031438935913


5175.635356471491

In [ ]:
df2 = df[['open', 'high', 'low', 'close', 'volume', 'signals']]
df2.reset_index(inplace=True)
df2['Index'] = df2.index
df2[['Index', 'datetime', 'open', 'high', 'low', 'close', 'volume', 'signals']].to_csv('logs.csv')